In [ ]:
# Data set split: 75-15-10

import os
import sys
import tensorflow.keras as keras
import tensorflow as tf
from tensorflow.python.keras.layers import Input, Dense, Flatten, Conv2D, MaxPooling2D, AveragePooling2D, Dropout
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers.merge import concatenate
from tensorflow.python.keras.regularizers import l2

In [ ]:
def inception_model(input, filters_1x1, filters_3x3_reduce, filters_3x3, 
                    filters_5x5_reduce, filters_5x5, filters_pool_proj):

    conv_1x1 = Conv2D(filters=filters_1x1, kernel_size=(1, 1), padding='same', 
                      activation='relu', kernel_regularizer=l2(0.01))(input)
    
    conv_3x3_reduce = Conv2D(filters=filters_3x3_reduce, kernel_size=(1, 1), padding='same', 
                             activation='relu', kernel_regularizer=l2(0.01))(input)
    
    conv_3x3 = Conv2D(filters=filters_3x3, kernel_size=(3, 3), padding='same', 
                      activation='relu', kernel_regularizer=l2(0.01))(conv_3x3_reduce)
    
    conv_5x5_reduce  = Conv2D(filters=filters_5x5_reduce, kernel_size=(1, 1), padding='same', 
                              activation='relu', kernel_regularizer=l2(0.01))(input)
    
    conv_5x5 = Conv2D(filters=filters_5x5, kernel_size=(5, 5), padding='same', 
                      activation='relu', kernel_regularizer=l2(0.01))(conv_5x5_reduce)
    
    maxpool = MaxPooling2D(pool_size=(3, 3), strides=(1, 1), padding='same')(input)
    
    maxpool_proj = Conv2D(filters=filters_pool_proj, kernel_size=(1, 1), strides=(1, 1), padding='same', 
                          activation='relu', kernel_regularizer=l2(0.01))(maxpool)
    
    inception_output = concatenate([conv_1x1, conv_3x3, conv_5x5, maxpool_proj], axis=3)  # use tf as backend

    return inception_output

In [ ]:
def define_model(weight_path = None):
    input = Input(shape=(32, 32, 3))

    conv1_7x7_s2 = Conv2D(filters=64, kernel_size=(7, 7), strides=(2, 2), padding='same', 
                          activation='relu', kernel_regularizer=l2(0.01))(input)

    maxpool1_3x3_s2 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(conv1_7x7_s2)

    conv2_3x3_reduce = Conv2D(filters=64, kernel_size=(1, 1), padding='same', 
                              activation='relu', kernel_regularizer=l2(0.01))(maxpool1_3x3_s2)

    conv2_3x3 = Conv2D(filters=192, kernel_size=(3, 3), padding='same', 
                       activation='relu', kernel_regularizer=l2(0.01))(conv2_3x3_reduce)

    maxpool2_3x3_s2 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(conv2_3x3)

    inception_3a = inception_model(input=maxpool2_3x3_s2, filters_1x1=64, 
                                   filters_3x3_reduce=96, filters_3x3=128, 
                                   filters_5x5_reduce=16, filters_5x5=32, filters_pool_proj=32)

    inception_3b = inception_model(input=inception_3a, filters_1x1=128, 
                                   filters_3x3_reduce=128, filters_3x3=192,
                                   filters_5x5_reduce=32, filters_5x5=96, filters_pool_proj=64)

    maxpool3_3x3_s2 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(inception_3b)

    inception_4a = inception_model(input=maxpool3_3x3_s2, filters_1x1=192, 
                                   filters_3x3_reduce=96, filters_3x3=208, 
                                   filters_5x5_reduce=16, filters_5x5=48, filters_pool_proj=64)

    inception_4b = inception_model(input=inception_4a, filters_1x1=160,
                                   filters_3x3_reduce=112, filters_3x3=224,
                                   filters_5x5_reduce=24, filters_5x5=64, filters_pool_proj=64)

    inception_4c = inception_model(input=inception_4b, filters_1x1=128, 
                                   filters_3x3_reduce=128, filters_3x3=256,
                                   filters_5x5_reduce=24, filters_5x5=64, filters_pool_proj=64)

    inception_4d = inception_model(input=inception_4c, filters_1x1=112,
                                   filters_3x3_reduce=144, filters_3x3=288, 
                                   filters_5x5_reduce=32, filters_5x5=64, filters_pool_proj=64)

    inception_4e = inception_model(input=inception_4d, filters_1x1=256,
                                   filters_3x3_reduce=160, filters_3x3=320,
                                   filters_5x5_reduce=32, filters_5x5=128, filters_pool_proj=128)

    maxpool4_3x3_s2 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(inception_4e)

    inception_5a = inception_model(input=maxpool4_3x3_s2, filters_1x1=256,
                                   filters_3x3_reduce=160, filters_3x3=320,
                                   filters_5x5_reduce=32, filters_5x5=128, filters_pool_proj=128)

    inception_5b = inception_model(input=inception_5a, filters_1x1=384,
                                   filters_3x3_reduce=192, filters_3x3=384,
                                   filters_5x5_reduce=48, filters_5x5=128, filters_pool_proj=128)

    averagepool1_7x7_s1 = AveragePooling2D(pool_size=(7, 7), strides=(7, 7), padding='same')(inception_5b)

    drop1 = Dropout(rate=0.4)(averagepool1_7x7_s1)

    linear = Dense(units=4, activation='softmax', kernel_regularizer=l2(0.01))(Flatten()(drop1))
    last = linear


    model = Model(inputs=input, outputs=last)
    return model
    
if __name__ == '__main__':
    model = define_model()
    print(model.summary())


In [ ]:
sgd = tf.keras.optimizers.SGD(lr=1e-6, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd, loss='categorical_crossentropy' , metrics=['accuracy'])

In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator()
validation_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

training_set = train_datagen.flow_from_directory('D3/train/', 
                                                 target_size = (224,224), 
                                                 batch_size = 10, 
                                                 class_mode = 'categorical', 
                                                 shuffle = True)

validation_set = test_datagen.flow_from_directory('D3/validation/', 
                                            target_size = (224,224), 
                                            batch_size = 1, 
                                            class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('D3/test/', 
                                            target_size = (224,224), 
                                            batch_size = 1, 
                                            class_mode = 'categorical')

In [ ]:
checkpoint_path = "training_3/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(patience = 3)

In [ ]:
training = model.fit(training_set, 
                     epochs = 30, 
                     validation_data = validation_set, 
                     verbose = 1, 
                     callbacks = [cp_callback, early_stopping])

In [ ]:
results = model.evaluate(test_set)

In [ ]:
model.save('saved_model/GoogLeNet')

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model('saved_model/GoogLeNet')
tflite_model = converter.convert()
open("converted_model_googlenet.tflite", "wb").write(tflite_model)